In [5]:
import pandas as pd
import requests
import io
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder 
#Query to the World Bank API to retrieve inflation rate, unemployment rate, depttogdp ratio, and several other indicators. 

start_date = 2014
end_date = 2024

indicators = {
    'FP.CPI.TOTL.ZG': 'inflation_rate',
    'SL.UEM.TOTL.ZS': 'unemployment_rate',
    'GC.DOD.TOTL.GD.ZS': 'debt_to_gdp_ratio',
    'BX.KLT.DINV.WD.GD.ZS': 'foreign_direct_investment',
    'SP.POP.TOTL': 'population',
    'EN.POP.DNST': 'population_density_km',
    'SP.DYN.LE00.IN': 'years_existed',
    'VC.IHR.PSRC.P5': 'conflict_count',
    'NY.GDP.MKTP.CD': 'gdp_nominal',
    'NY.GDP.PCAP.PP.CD': 'gdp_ppp_per_capita',
    'NY.GDP.PCAP.CD': 'gdp_nominal_per_capita',
    'SI.POV.GINI': 'gini_coefficient',
    'HD.HCI.OVRL': 'human_development_index',
    'DT.DOD.DECT.CD': 'gross_external_debt',
}
base_url = "http://api.worldbank.org/v2/country/all/indicator/"
indicator_data = []

for indicator, name in indicators.items():
    url = f"{base_url}{indicator}?date={start_date}:{end_date}&format=json&per_page=20000"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1 and 'country' in data[1][0]:
            records = [
                {
                    'Country': entry['country']['value'],
                    'Country Code': entry['countryiso3code'],
                    'Year': entry['date'],
                    name: entry['value']
                }
                for entry in data[1] if entry['value'] is not None
            ]
            indicator_data.append(pd.DataFrame(records))

if indicator_data:
    df = indicator_data[0]
    for additional_df in indicator_data[1:]:
        df = pd.merge(df, additional_df, on=['Country', 'Country Code', 'Year'], how='outer')
    df.sort_values(by=['Country', 'Year'], ascending=[True, False], inplace=True)
    df.to_csv('world_bank_economic_data.csv', index=False)
    print(df.head())
else:
    print("No data")


#Appended to add Happiness Index Analysis 

# Load the World Happiness Report
happiness_data = pd.read_excel("DataForTable2.1.xls")

# Print the column names
print(happiness_data.columns)

# Merge World Bank data with Happiness data
combined_data = pd.merge(df, happiness_data, left_on='Country', right_on='Country name', how='inner')
print(combined_data.head())  

# Drop unnecessary columns
combined_data = combined_data.drop(columns=['Year', 'Country', 'Country name'], errors='ignore')

# Remove rows with missing values
combined_data = combined_data.dropna()

# Define the feature set and targets
X = combined_data.drop(columns=['Life Ladder', 'Country Code'])  
X = X.select_dtypes(include=[float, int])  
y = combined_data['Life Ladder']

# Split the dataset into training and testing sets 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Adding on Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

rf_r2_score = r2_score(y_test, y_pred_rf)

#Evalve using R2 score and display feature coefficients for both ML methods
print("Linear Regression:", r2_score(y_test, y_pred))
print("Feature Coefficients:", model.coef_)
print("Linear Regression R2 Score:", r2_score(y_test, y_pred))
print("Linear Regression Coefficients:", model.coef_)
print("Random Forest R2 Score:", rf_r2_score)

       Country Country Code  Year  inflation_rate  unemployment_rate  \
9  Afghanistan          AFG  2023             NaN             14.386   
8  Afghanistan          AFG  2022             NaN             14.100   
7  Afghanistan          AFG  2021             NaN             11.934   
6  Afghanistan          AFG  2020             NaN             11.710   
5  Afghanistan          AFG  2019        2.302373             11.224   

   debt_to_gdp_ratio  foreign_direct_investment  population  \
9                NaN                        NaN  42239854.0   
8                NaN                        NaN  41128771.0   
7                NaN                   0.144401  40099462.0   
6                NaN                   0.064994  38972230.0   
5                NaN                   0.124496  37769499.0   

   population_density_km  years_existed  conflict_count   gdp_nominal  \
9                    NaN            NaN             NaN           NaN   
8                    NaN         62.879   